## Funktionen

> Potenzielles TODO: Zu jeder hier definierten Funktion schreiben in welchem Notebook sie verwendet werden.

Neben den Importen werden auch drei Funktionen definiert, die über die verwendeten Notebooks hinweg benötigt werden. Für die Verwendung muss ebenfalls dieses Notebook zuerst ausgeführt werden.

> %run Util/01_functions.ipynb

> TODO: Darstellungsweise der Parameter und Rückgabewerte evtl. anpassen
> TODO: Begriff "situation" hat ihm im Verlauf der Arbeit nicht verstanden

Als Nächstes wird eine Funktion definiert, die die Suche innerhalb einer ``sequence`` ermöglicht. Diese ``sequence`` ist eine Liste bestehend aus Mengen. In diesen Mengen stehen die Spielsituationen in einer der drei zuvor erwähnten Darstellungsweisen. Bei der ``situation`` muss darauf geachtet werden, dass sie die gleiche Darstellung besitzt wie die Elemente in der ``sequence``. Sofern die Funktion ``find_situation_in_sequence`` die Spielsituation in einer der Mengen findet, wird der Mengenindex der Liste zurückgegeben. Andernfalls lautet der Rückgabewert -1.

In [ ]:
def find_situation_in_sequence(situation, sequence):
    for i in range(len(sequence)):
        if situation in sequence[i]:
            return i
    return -1